<a href="https://colab.research.google.com/github/ryo-nakajima/the_effect_test/blob/master/uth_bert_part1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install tensorflow
!pip install mecab-python3
!pip install jaconv
!pip install neologdn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 581.7/581.7 kB 23.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for jaconv: filename=jaconv-0.4.0-py3-none-any.whl size=18229 sha256=144745be2ca0e6e250364234aac8c0e42ed6669240cb5fe7a072f99af93eddc3
  Stored in directory: /root/.cache/pip/wheels/20/95/99/94e8d7545125181756857f6b1fc085ed4e0811ad9be7321af7
Successfully built jaconv
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.6/144.6 kB 9.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for neologdn: filename=neologdn-0.5.3-cp310-cp310-linux_x86_64.whl size=223873 sha256=f0cd3ea554ace0005050ada9b7f4db8f4545d0fc1fdac24b3130a304c6403db4
  Stored in directory: /root/.cache/pip/wheels/f3/06/cd/3216b1007983e3cde801f6376c641617c37f4f520f9faed644
Successfully built neologdn


In [2]:
import os
# UTH-BERTモデルのダウンロード
!wget https://ai-health.m.u-tokyo.ac.jp/labweb/dl/uth_bert/UTH_BERT_BASE_512_MC_BPE_WWM_V25000_352K_pytorch.zip
!unzip UTH_BERT_BASE_512_MC_BPE_WWM_V25000_352K_pytorch.zip
!rm UTH_BERT_BASE_512_MC_BPE_WWM_V25000_352K_pytorch.zip
!git clone https://github.com/jinseikenai/uth-bert.git

# MeCabとNEologdのインストール
!apt install mecab libmecab-dev mecab-ipadic-utf8 file
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git
!mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -a -y
os.environ['MECABRC'] = "/etc/mecabrc"


# 万病辞書のダウンロード
!wget http://sociocom.jp/~data/2018-manbyo/data/MANBYO_201907_Dic-utf8.dic

--2024-11-21 03:20:04--  https://ai-health.m.u-tokyo.ac.jp/labweb/dl/uth_bert/UTH_BERT_BASE_512_MC_BPE_WWM_V25000_352K_pytorch.zip
Resolving ai-health.m.u-tokyo.ac.jp (ai-health.m.u-tokyo.ac.jp)... 163.43.80.52
Connecting to ai-health.m.u-tokyo.ac.jp (ai-health.m.u-tokyo.ac.jp)|163.43.80.52|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 393100779 (375M) [application/zip]
Saving to: ‘UTH_BERT_BASE_512_MC_BPE_WWM_V25000_352K_pytorch.zip’

UTH_BERT_BASE_512_M 100%[===================>] 374.89M  26.8MB/s    in 15s     

2024-11-21 03:20:20 (24.4 MB/s) - ‘UTH_BERT_BASE_512_MC_BPE_WWM_V25000_352K_pytorch.zip’ saved [393100779/393100779]

Archive:  UTH_BERT_BASE_512_MC_BPE_WWM_V25000_352K_pytorch.zip
  inflating: UTH_BERT_BASE_512_MC_BPE_WWM_V25000_352K/config.json  
  inflating: UTH_BERT_BASE_512_MC_BPE_WWM_V25000_352K/vocab.txt  
  inflating: UTH_BERT_BASE_512_MC_BPE_WWM_V25000_352K/pytorch_model.bin  
Cloning into 'uth-bert'...
remote: Enumerating objects: 108, d

MeCab、NEologd、万病辞書を使ったトークナイザの設定

In [3]:
import MeCab
import subprocess

# NEologdのパスを取得
cmd = 'echo `mecab-config --dicdir`"/mecab-ipadic-neologd"'
neologd_dic_dir_path = subprocess.check_output(cmd, shell=True).decode('utf-8').strip()

# MeCabを初期化
mecab = MeCab.Tagger(f"-d {neologd_dic_dir_path}")

# 万病辞書の設定
manbyo_dic_path = "MANBYO_201907_Dic-utf8.dic"

# トークナイザ関数の定義
def tokenize_with_mecab(text):
    text = mecab.parse(text)
    return text.splitlines()

PyTorchでUTH-BERTモデルをロード

In [4]:


!ls ./UTH_BERT_BASE_512_MC_BPE_WWM_V25000_352K

config.json  pytorch_model.bin	vocab.txt


In [5]:
from transformers import BertTokenizer, BertModel

# モデルのパスを指定
model_path = "./UTH_BERT_BASE_512_MC_BPE_WWM_V25000_352K"

# トークナイザとモデルをロード
tokenizer = BertTokenizer.from_pretrained(model_path)
model = BertModel.from_pretrained(model_path)

# モデル確認
print(model)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(25000, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

トークン化とモデルでの推論

In [6]:
# サンプルテキスト
sample_text = "東京大学医学部附属病院で研究しています。"

# MeCabで形態素解析を実行
tokens = tokenize_with_mecab(sample_text)
print("MeCab Tokens:", tokens)

# MeCabのトークンをHugging Faceトークナイザに渡す
bert_tokens = tokenizer(tokens, is_split_into_words=True, return_tensors="pt")
print("BERT Tokenized Inputs:", bert_tokens)

# モデルで推論を実行
outputs = model(**bert_tokens)

# 隠れ層の出力を確認
last_hidden_states = outputs.last_hidden_state
print("Last Hidden States Shape:", last_hidden_states.shape)

MeCab Tokens: ['東京大学医学部附属病院\t名詞,固有名詞,組織,*,*,*,東京大学医学部附属病院,トウキョウダイガクイガクブフゾクビョウイン,トーキョーダイガクイガクブフゾクビョーイン', 'で\t助詞,格助詞,一般,*,*,*,で,デ,デ', '研究\t名詞,サ変接続,*,*,*,*,研究,ケンキュウ,ケンキュー', 'し\t動詞,自立,*,*,サ変・スル,連用形,する,シ,シ', 'て\t助詞,接続助詞,*,*,*,*,て,テ,テ', 'い\t動詞,非自立,*,*,一段,連用形,いる,イ,イ', 'ます\t助動詞,*,*,*,特殊・マス,基本形,ます,マス,マス', '。\t記号,句点,*,*,*,*,。,。,。', 'EOS']
BERT Tokenized Inputs: {'input_ids': tensor([[    2,  6818, 22661,   694,  2574,    76,  2574,   163,     1, 22508,
          1591,   273,   953, 22207,     1, 13502,  2488,   953, 22207,     1,
         10293, 13922,     1,     1,     1,     1,     1,     1,     1,  6818,
         22661,   694,  2574,    76,  2574,   163,     1, 22508,  1591,   273,
             1,  4644,  3031,  2734, 15769,  3031,  7345, 21556,  2197, 21556,
          2885,  2885,  4345,  2451,  3022, 15769, 14715,  2368,     1,  4644,
          2023,  2734, 15769,  2023,  7345, 21556,  2197, 21556,  2885,  2885,
          4345,  2451,  3022, 15769,  2023,  2373,    13,  6515, 22207,     1,


In [33]:
import torch
import torch.nn.functional as F
from transformers import BertTokenizer, BertModel

# モデルのパス
model_path = "./UTH_BERT_BASE_512_MC_BPE_WWM_V25000_352K"

# トークナイザとモデルをロード
tokenizer = BertTokenizer.from_pretrained(model_path)
model = BertModel.from_pretrained(model_path)

# MeCabで形態素解析する関数 (仮定: MeCabは事前に設定済み)
def tokenize_with_mecab(text):
    import MeCab
    mecab = MeCab.Tagger("-Owakati")
    tokens = mecab.parse(text).strip().split()
    return tokens

# BERTを用いて特徴ベクトルを取得する関数
def get_cls_embedding(text, model, tokenizer):
    # MeCabで形態素解析
    tokens = tokenize_with_mecab(text)

    # Hugging Faceトークナイザでトークナイズ
    encoded = tokenizer(tokens, is_split_into_words=True, return_tensors="pt")

    # モデルを推論モードに設定
    model.eval()
    with torch.no_grad():
        outputs = model(**encoded)

    # [CLS]トークンの埋め込みを取得 (通常、位置0に対応)
    cls_embedding = outputs.last_hidden_state[:, 0, :]  # Shape: (1, hidden_size)
    return cls_embedding.squeeze(0)  # Shape: (hidden_size,)

# コサイン類似度を計算する関数
def calculate_cosine_similarity(embedding1, embedding2):
    similarity = F.cosine_similarity(embedding1, embedding2, dim=0)
    return similarity.item()

# テキスト
sample_text1 = "東京大学医学部附属病院で研究しています。"
sample_text2 = "京都大学医学部附属病院で研修医をしています。専門は脳外科です"

# 特徴ベクトルを取得
embedding1 = get_cls_embedding(sample_text1, model, tokenizer)
embedding2 = get_cls_embedding(sample_text2, model, tokenizer)

# 類似度を計算
similarity = calculate_cosine_similarity(embedding1, embedding2)

# 結果を表示
print(f"Cosine Similarity between text1 and text2: {similarity}")

Cosine Similarity between text1 and text2: 0.9129119515419006


PyTorch-Lightningを使用してUTH-BERTモデルを文分類タスクにファインチューニングする

ライブラリーインストール

In [7]:
!pip install pytorch-lightning transformers torch scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 926.4/926.4 kB 46.2 MB/s eta 0:00:00


データ準備

In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split

# サンプルデータセット
data = {
    "text": [
        "東京大学で医療AIを研究しています。",
        "医療AIは日本の未来に大きく貢献します。",
        "医療に関する研究は非常に重要です。",
        "私はスポーツが好きです。",
        "昨日はサッカーの試合を見ました。",
        "野球も好きですが、サッカーが一番です。",
    ],
    "label": [1, 1, 1, 0, 0, 0],  # 医療関連: 1, その他: 0
}

df = pd.DataFrame(data)

# データを分割
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df["text"], df["label"], test_size=0.2, random_state=42
)


トークナイザー定義

In [9]:
# MeCabで形態素解析
def mecab_tokenize(text, mecab):
    parsed = mecab.parse(text).strip()
    tokens = [line.split('\t')[0] for line in parsed.splitlines() if line != "EOS"]
    return tokens

# Hugging Faceトークナイザでエンコード
def tokenize_data_with_mecab(texts, labels, tokenizer, max_length=128):
    all_encodings = []
    for text in texts:
        # MeCabで形態素解析
        mecab_tokens = mecab_tokenize(text, mecab)
        # トークナイザでエンコード
        encodings = tokenizer(
            mecab_tokens,
            is_split_into_words=True,
            truncation=True,
            padding="max_length",
            max_length=max_length,
            return_tensors="pt"
        )
        all_encodings.append(encodings)

    # バッチ単位に変換
    input_ids = torch.cat([e["input_ids"] for e in all_encodings], dim=0)
    attention_mask = torch.cat([e["attention_mask"] for e in all_encodings], dim=0)
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask
    }, torch.tensor(labels)


In [10]:
import torch

# サンプルデータ
train_texts = ["東京大学医学部附属病院で研究しています。", "糖尿病は日本の大きな課題です。"]
train_labels = [1, 0]  # ラベル（例: 1=ポジティブ, 0=ネガティブ）

val_texts = ["医学研究が進歩しています。", "スポーツは健康に良いです。"]
val_labels = [1, 0]

# トークナイズ
train_encodings, train_labels = tokenize_data_with_mecab(train_texts, train_labels, tokenizer)
val_encodings, val_labels = tokenize_data_with_mecab(val_texts, val_labels, tokenizer)

# 結果確認
print("Train Encodings:", train_encodings)
print("Train Labels:", train_labels)


Train Encodings: {'input_ids': tensor([[    2,  6818, 22661,   694,  2574,    76,  2574,   163,     1, 22508,
          1591,   273,    13, 21476,     1,    16,    13,    66,    57,     6,
             3,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0

In [11]:
def tokenize_data_with_mecab(texts, labels, tokenizer, max_length=128):
    all_encodings = []
    for text in texts:
        mecab_tokens = mecab_tokenize(text, mecab)
        encodings = tokenizer(
            mecab_tokens,
            is_split_into_words=True,
            truncation=True,
            padding="max_length",
            max_length=max_length,
            return_tensors="pt"
        )
        all_encodings.append(encodings)

    input_ids = torch.cat([e["input_ids"] for e in all_encodings], dim=0)
    attention_mask = torch.cat([e["attention_mask"] for e in all_encodings], dim=0)
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask
    }, torch.tensor(labels)

Datasetクラスの定義:

In [12]:
from torch.utils.data import Dataset
class TextDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item["labels"] = self.labels[idx]
        return item


DataLoaderの準備:

In [13]:
from torch.utils.data import DataLoader
train_dataset = TextDataset(train_encodings, train_labels)
val_dataset = TextDataset(val_encodings, val_labels)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8)


PyTorch-Lightningを用いたモデルの学習

In [14]:
# DataLoaderの確認
for batch in train_loader:
    print(batch)
    break

{'input_ids': tensor([[    2,  2264,   159,  1591,    15,    38,   200,    10,   694,   197,
          2069, 11578, 19843,    13,  2711,     6,     3,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,  

モデルの定義

In [29]:
import torch
from torch.nn import CrossEntropyLoss
from transformers import BertModel
import pytorch_lightning as pl
from torch.optim import AdamW

class UTHBERTClassifier(pl.LightningModule):
    def __init__(self, model_name, num_classes, learning_rate=2e-5):
        super().__init__()
        self.bert = BertModel.from_pretrained(model_name)
        self.classifier = torch.nn.Linear(self.bert.config.hidden_size, num_classes)
        self.loss_fn = CrossEntropyLoss()
        self.learning_rate = learning_rate

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        logits = self.classifier(outputs.last_hidden_state[:, 0, :])  # CLSトークンの出力を使用
        return logits

    def training_step(self, batch, batch_idx):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["labels"]

        logits = self(input_ids, attention_mask)
        loss = self.loss_fn(logits, labels)
        self.log("train_loss", loss, on_epoch=True, on_step=True, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["labels"]

        logits = self(input_ids, attention_mask)
        loss = self.loss_fn(logits, labels)

        preds = torch.argmax(logits, dim=1)
        acc = (preds == labels).float().mean()

        self.log("val_loss", loss, on_epoch=True, prog_bar=True)
        self.log("val_acc", acc, on_epoch=True, prog_bar=True)
        return {"val_loss": loss, "val_acc": acc}

    def configure_optimizers(self):
        return AdamW(self.parameters(), lr=self.learning_rate)


学習/検証ステップ

In [16]:
def training_step(self, batch, batch_idx):
    input_ids = batch["input_ids"]
    attention_mask = batch["attention_mask"]
    labels = batch["labels"]

    logits = self(input_ids, attention_mask)
    loss = self.loss_fn(logits, labels)

    self.log("train_loss", loss, on_epoch=True, on_step=True, prog_bar=True)
    return loss

In [17]:
def validation_step(self, batch, batch_idx):
    input_ids = batch["input_ids"]
    attention_mask = batch["attention_mask"]
    labels = batch["labels"]

    logits = self(input_ids, attention_mask)
    loss = self.loss_fn(logits, labels)

    preds = torch.argmax(logits, dim=1)
    acc = (preds == labels).float().mean()

    self.log("val_loss", loss, on_epoch=True, prog_bar=True)
    self.log("val_acc", acc, on_epoch=True, prog_bar=True)
    return {"val_loss": loss, "val_acc": acc}


オプティマイザの設定

In [18]:
def configure_optimizers(self):
    return AdamW(self.parameters(), lr=self.learning_rate)


PyTorch-LightningのTrainerを使用した学習

In [30]:
# モデルをインスタンス化
model_name = "bert-base-uncased"  # 必要に応じて日本語モデルを指定
num_classes = 2  # クラス数（医療関連とその他の分類）
model = UTHBERTClassifier(model_name, num_classes)

In [31]:
print(type(model))

<class '__main__.UTHBERTClassifier'>


In [32]:
trainer = pl.Trainer(
    max_epochs=3,
    accelerator="gpu" if torch.cuda.is_available() else "cpu",
    devices=1 if torch.cuda.is_available() else None,
)
trainer.fit(model, train_loader, val_loader)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name       | Type             | Params | Mode 
--------------------------------------------------------
0 | bert       | BertModel        | 109 M  | eval 
1 | classifier | Linear           | 1.5 K  | train
2 | loss_fn    | CrossEntropyLoss | 0      | train
--------------------------------------------------------
109 M     Trainable params
0         Non-trainable params
109 M     Total params
437.935   Total estimated model params size (MB)
2         Modules in train mode
228       Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/loops/fit_loop.py:298: The number of training batches (1) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=3` reached.
